<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Supervised Learning - Decision Tree - Titanic
</b></font> </br></p>

---


# 0  | Install & Import
***

In [1]:
# Install
!uv pip install git+https://github.com/parrt/dtreeviz.git

Using Python 3.12.12 environment at: /usr
Resolved 21 packages in 38.36s
Prepared 1 package in 388ms
Installed 1 package in 2ms
 + dtreeviz==2.3.2 (from git+https://github.com/parrt/dtreeviz.git@9680ac83b778124c9e21d018261144fbcdde5265)


In [2]:
# Import
from pandas import read_excel, DataFrame

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, export_graphviz
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

import plotly.express as px
import plotly.subplots as sp

import graphviz
import dtreeviz

In [3]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# Matplotlib-spezifische Warnungen unterdrücken
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)

# 1  | Understand
***

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
📒 Anwendungsfall
</font></p>

Dies ist der legendäre Titanic ML-Wettbewerb – die beste erste Herausforderung, um in ML-Modellierung einzutauchen.

Die Aufgabe ist einfach: Verwenden Sie maschinelles Lernen, um ein Modell zu erstellen, das vorhersagt, welche Passagiere den Schiffbruch der Titanic überlebt haben.

[Titanic Org](https://www.encyclopedia-titanica.org/)

[DataSet](https://www.openml.org/search?type=data&status=active&id=41265)

[Info](https://www.kaggle.com/competitions/titanic/data)



**Datenfelder:**   
+ Age: Alter
+ Fare: Ticketpreis
+ Sex: Geschlecht (0 = männlich, 1 = weiblich)
+ sibsp: Der Datensatz definiert Familienbeziehungen auf diese Weise ... Geschwister = Bruder, Schwester, Stiefbruder, Stiefschwester Ehepartner = Ehemann, Ehefrau (Geliebte und Verlobte wurden ignoriert)
+ parch: Der Datensatz definiert Familienbeziehungen auf diese Weise ... Elternteil = Mutter, Vater Kind = Tochter, Sohn, Stieftochter, Stiefsohn. Einige Kinder reisten nur mit einem Kindermädchen, daher ist für sie Parch=0
+ Pclass: Passagierklasse, 1.- 3. Klasse
+ Embarked: Hafen der Einschiffung

In [4]:
df = read_excel(
    "https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/titanic.xlsx",
    usecols=["pclass", "survived", "sex", "age", "sibsp", "parch"],
)

In [5]:
data = df.copy()
target = data.pop("survived")

<p><font color='black' size="5">
🔎 EDA (Exploratory Data Analysis) mit Pandas
</font></p>

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1307 entries, 0 to 1306
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     1307 non-null   float64
 1   sex     1307 non-null   object 
 2   sibsp   1307 non-null   int64  
 3   parch   1307 non-null   int64  
 4   pclass  1307 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 51.2+ KB


In [7]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1307.0,29.471821,12.881592,0.17,22.0,28.0,35.0,80.0
sibsp,1307.0,0.499617,1.042273,0.00,0.0,0.0,1.0,8.0
parch,1307.0,0.385616,0.866092,0.00,0.0,0.0,0.0,9.0
pclass,1307.0,2.296863,0.836942,1.00,2.0,3.0,3.0,3.0


In [8]:
data.groupby("sex").count()

,age,sibsp,parch,pclass
sex,,,,
female,464,464,464,464
male,843,843,843,843


In [9]:
target.value_counts()

,count
survived,
0,967
1,340


In [10]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1307.0,29.471821,12.881592,0.17,22.0,28.0,35.0,80.0
sibsp,1307.0,0.499617,1.042273,0.00,0.0,0.0,1.0,8.0
parch,1307.0,0.385616,0.866092,0.00,0.0,0.0,0.0,9.0
pclass,1307.0,2.296863,0.836942,1.00,2.0,3.0,3.0,3.0


# 2 | Prepare

---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Datentyp ermitteln/ändern</br>
✅ Train-Test-Split durchführen</br>
✅ Nicht benötigte Features löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>

<font color='black' size="5">
Datentyp ermitteln
</font>

In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<font color='black' size="5">
Train-Test-Set
</font>

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.20, stratify=target, random_state=42)

<font color='black' size="5">
Missing Values - numerische Features
</font>

In [ ]:
imputer_num = SimpleImputer(strategy="mean")
imputer_num.fit(data_train[num_col])
data_train[num_col] = imputer_num.transform(data_train[num_col])
data_test[num_col] = imputer_num.transform(data_test[num_col])

<font color='black' size="5">
Kodierung
</font>

In [ ]:
encoder = OrdinalEncoder()
encoder.fit(data_train[cat_col])
data_train[cat_col] = encoder.transform(data_train[cat_col])
data_test[cat_col] = encoder.transform(data_test[cat_col])

# 3 | Modeling
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellauswahl treffen</br>
✅ Pipeline erweitern/konfigurieren</br>
✅ Training durchführen</br>
✅ Hyperparameter Tuning</br>
✅ Cross-Valdiation</br>
✅ Bootstrapping</br>
✅ Regularization</br>

<p><font color='black' size="5">
🏃 Modellauswahl & Training
</font></p>

In [ ]:
model = DecisionTreeClassifier(max_depth=3, random_state=42)

In [ ]:
model.fit(data_train, target_train)

# 4 | Evaluate
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Prognose (Train, Test) erstellen</br>
✅ Modellgüte prüfen</br>
✅ Residuenanalyse erstellen</br>
✅ Feature Importance/Selektion prüfen</br>
✅ Robustheitstest erstellen</br>
✅ Modellinterpretation erstellen</br>
✅ Sensitivitätsanalyse erstellen</br>
✅ Kommunikation (Key Takeaways)</br>

<p><font color='black' size="5">
🔭 Prognose
</font></p>

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

<p><font color='black' size="5">
📱 Confusion Matrix
</font></p>

In [ ]:
conf_matrix = confusion_matrix(target_test, target_test_pred)
display_labels_ = ["Not Survived", "Survived"]
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=display_labels_)
disp.plot(cmap="Blues")

In [ ]:
print(classification_report(target_test, target_test_pred, target_names=display_labels_))

<p><font color='black' size="5">
🎯 Accuracy
</font></p>

In [ ]:
acc_train = accuracy_score(target_train, target_train_pred) * 100
print(f"Modell: {model} -- Train -- Accuracy: {acc_train:5.2f}")

In [ ]:
acc_test = accuracy_score(target_test, target_test_pred) * 100
print(f"Modell: {model} -- Test -- Accuracy: {acc_test:5.2f}%")

<p><font color='black' size="5">
🎲Aufbau Analysewürfel
</font></p>

In [ ]:
# Übernahme der Testdaten
cube = data_test.copy()
cube.reset_index(inplace=True)

In [ ]:
# Übernahme Target real & predict
cube["real"] = DataFrame(target_test.values, columns=["real"])
cube["predict"] = DataFrame(target_test_pred, columns=["predict"])

<p><font color='black' size="5">
📊 Visualisierung real vs predict
</font></p>

In [ ]:
# Histogramm
title_ = "Histogramm real vs predict"
fig = px.histogram(cube, x=["real", "predict"], nbins=2, title=title_)
fig.update_layout(barmode="group", bargap=0.1, width=600, height=600)
fig.show()

<p><font color='black' size="5">
📛 Fehlerhafte Vorhersagen
</font></p>

In [ ]:
# real <> predict
cube[cube.real != cube.predict].describe().T

In [ ]:
cube[cube.real != cube.predict]

<p><font color='black' size="5">
📝 Einzelne Vorhersage
</font></p>

In [ ]:
# 2 neue Datensätze werden zur Prognose an das Modell übergeben: Rose & Jack (Winslet/diCaprio)
new_data = {
    "age": [22, 23],
    "sex": [0, 1],
    "sibsp": [0, 0],
    "parch": [0, 0],
    "pclass": [1, 3]
}
new = DataFrame(new_data)

In [ ]:
# Vorhersage erstellen Rose & Jack
model.predict(new)

<p><font color='black' size="5">
❗Feature Importance
</font></p>

In [ ]:
title_ = "Feature Importance Titanic"
px.bar(
    x=model.feature_importances_, y=data.columns, title=title_, width=800, height=600
).update_yaxes(categoryorder="total ascending")

<p><font color='black' size="5">
🌳 Darstellen des Entscheidungsbaums
</font></p>

Erstellen einen Textbericht, der die Regeln eines Entscheidungsbaums zeiget.
Für die Anzeige class_name muss ein Update auf sklearn 1.4 erfolgen.

<p><font color='blue' size="4">
Tree Rules
</font></p>

In [ ]:
tree_rules = export_text(
    model,
    feature_names=list(model.feature_names_in_),
    class_names=display_labels_,
    spacing=7,
    decimals=1,
    show_weights=True,
)
print(tree_rules)

<p><font color='blue' size="4">

Baumstruktur `graphviz`
</font></p>

In [ ]:
# DOT data
dot_data = export_graphviz(
    model,
    out_file=None,
    feature_names=data.columns,
    class_names=display_labels_,
    filled=True,
)

Aufbau Knoten im Entscheidungsbaum

In [ ]:
# Draw graph
graph = graphviz.Source(dot_data, format="png")
graph

<p><font color='blue' size="4">

Baumstruktur `dtreeviz`

</font></p>

In [ ]:
viz_model = dtreeviz.model(
    model,
    data_test,
    target_test,
    target_name="survived",
    class_names=display_labels_,
    feature_names=data.columns,
)

In [ ]:
# Top-Down
viz_model.view(scale=1.5, fontname="Monospace")

In [ ]:
# Left-Right
viz_model.view(scale=1.5, orientation="LR", fontname="Monospace")

In [ ]:
# local Explanation - total
one = data_test.iloc[13]
viz_model.view(x=one, scale=1.2,fontname="Monospace")

In [ ]:
# local Explanation - single
viz_model.view(x=one, scale=1.2, show_just_path=True, fontname="Monospace")

In [ ]:
# Save
tree_img = viz_model.view(scale=1.2, fontname="Monospace")
tree_img.save("tree.svg")

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>